Machine Learning - Semester Project 
------------------------------------------------------------

Done by Wania Naeem (22I-2369) and Emaan Ali (22I-2325)

This is the first part of the project - DATA PREPROCESSING!!!

-> We are downloading the Fashion MNIST dataset using tensorflow/ keras